In [ ]:
#importando as dependencias necessarias
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
#formatando o arquivo de logger para ser compativel com o pandas
#obs: o arquivo de logger sera sobreescrito (podendo nao ser necesssario executar essa celula para determinado arquivo)
with open('data/loggerECG.json', 'r+') as file:
    dic = file.read()
    f = json.loads(dic)
    for key in f:
        f[key] = list(f[key][0][1:-1].split(', '))

    for key in f:
        d = {}
        for i in range(len(f[key])):
            d[i] = f[key][i]
        f[key] = d
    file.seek(0)
    file.write(json.dumps(f))
    file.truncate()

In [ ]:
#definicao de variaveis
items = ['battery', 'size', 'pressure', 'lux', 'heartbeat',
         'ecg', 'ppg', 'saturation', 'accelerometerZ', 'accelerometerX',
         'accelerometerY', 'gyroscopeZ', 'gyroscopeX', 'gyroscopeY']
units = {'battery': 'bateria (%)',
             'size': 'armazenamento (MB)',
             'pressure': 'pressao (hPa)',
             'lux': 'luminosidade (lux)',
             'heartbeat': 'taxa de batimentos (bpm)',
             'ecg': 'ecg cru (mV)',
             'ppg': 'ppg cru (mV)',
             'saturation': 'oxigenacao crua (mV)',
             'accelerometerZ': 'aceleracao (m/s²)',
             'accelerometerX': 'aceleracao (m/s²)',
             'accelerometerY': 'aceleracao (m/s²)',
             'gyroscopeZ': 'giroscopio (rad/s)',
             'gyroscopeX': 'giroscopio (rad/s)',
             'gyroscopeY': 'giroscopio (rad/s)'}

In [ ]:
#removendo o id do smartwatch do dataframe
df = pd.read_json('data/loggerECG.json')
df.drop(['ID_Smartwatch'], axis=1, inplace=True)
#print(df)

In [ ]:
#visao geral dos dados (quantidade, media, desvio padrao, minimo, q1, mediana, q3 e maximo)
#df.head()
df.describe()

In [ ]:
#checando a existencia de valores nulos/faltantes
#df.isna().sum()
df.info()

In [ ]:
#substituindo os valores faltantes pela media da coluna (caso nao existam nao e necessario executar essa celula)
#obs: apenas valido para valores numericos
num_col = [] # lista com as colunas que possuem valores faltantes
for col in num_col:
    df[col]=pd.to_numeric(df[col])
    df[col].fillna(df[col].mean(), inplace=True)
df.isna().sum()

In [ ]:
y = list(df.heartbeat)
plt.boxplot(y)